In [62]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import re
from PIL import Image
import html
from markdownify import markdownify 
import json

# endpoint_url = "https://query.wikidata.org/sparql"

# query_img ="""
# SELECT DISTINCT ?artwork ?artworkLabel ?image ?width ?height ?artworkDescription (YEAR(?incep) AS ?year) (STRAFTER(STR(?artwork), "/entity/") AS ?qid) WHERE {
#   ?artwork wdt:P31 wd:Q3305213;
#     wdt:P170 wd:Q41406;
#     wdt:P18 ?image;
#     wdt:P2048 ?height;
#     wdt:P2049 ?width;
#     wdt:P571 ?incep.
#   FILTER(?width > ?height)
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "en,[AUTO_LANGUAGE]". }
# }
# LIMIT 25
# """

endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"


query_img ="""
PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT DISTINCT ?itemLabel ?itemDescr ?imgItem ?imgUrl ?publishDate
WHERE
{
  ?imgItem cpsp:P107 ?urlStatement. 
  ?urlStatement cpsps:P107 ?imgUrl. 
  ?imgItem cpsp:P60 ?dateStatement. 
  ?dateStatement cpsps:P60 ?publishDate. 
  ?imgItem cpsp:P6 ?partOfStatement.
  ?partOfStatement cpsps:P6 ?partOfItem.
  <placeholder> 

  FILTER (datatype(?publishDate) = xsd:edtf)
  
  SERVICE wikibase:label {
      bd:serviceParam wikibase:language "en,de".
      ?imgItem rdfs:label ?itemLabel.
      ?imgItem schema:description ?itemDescr.
    }
}
"""

def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_delay(date):
    try:
        date = datetime.datetime.strptime(date, '%a, %d %b %Y %H:%M:%S GMT')
        timeout = int((date - datetime.datetime.now()).total_seconds())
    except ValueError:
        timeout = int(date)
    return timeout


def get_img(partOfItem_id):
    print("")
    print("<link rel='stylesheet' href='./section-fullimg.css'>")

    q = ""
    print("<div class='allcontent'>")
    print("<div class='navig'>")

    if partOfItem_id:
        q = query_img.replace("<placeholder>", "?partOfStatement cpsps:P6 cps:"+partOfItem_id+".")
    else:
        q = query_img.replace("<placeholder>","")
    results_img = run_query(endpoint_url, q)


    results = []
    for item in results_img["results"]["bindings"]:
        image = item["imgUrl"]["value"] if 'imgUrl' in item else ''
        title = item["itemLabel"]["value"] if 'itemLabel' in item else ''

        filename = image.split('/')[-1].split('.')
        url_end = re.sub('[^0-9a-zA-Z]+', '_', filename[-2]) 
        image_path = './images/' + url_end + '.' + filename[-1] 
        results.append({
            "year": item.get("publishDate", {}).get("value", "Unknown"),
            "name": item.get("itemLabel", {}).get("value", "No title"),
            "link": item.get("artwork", {}).get("value", "#"),
            "alt": item.get("itemDescr", {}).get("value", "Painting"),
            "image": item["imgUrl"]["value"],
            "qid": item.get("qid", {}).get("value", ""),
            "width": item.get("width", {}).get("value", "?"),
            "height": item.get("height", {}).get("value", "?")
        })
        
        print("""
::: {.content-visible when-format='pdf'}
        """)
        print(f"""
\\begin{{figure}}[H]
\\centering
\\includegraphics[height=10cm]{{{image_path}}}
\\caption{{{title.replace('&', '+')}}}
\\label{{fig:{{{image}}}}}
\\end{{figure}}

```{{=latex}}
\\clearpage
```
        """)

        print('''
:::
        ''')

        print("""
::: {.content-visible when-format='docx'}
        """)

        print(f"""
![{title}]({image_path})
        """)

        print('''
:::
        ''')

    json_data = json.dumps(results)

    print("<input type='button' class='btn btn-link arrow arrowleft' onclick='openPrev()' value='&#10092;'/>")

    print(f'<img id="img_full" src="" alt="">')

    print("<input type='button' class='btn btn-link arrow arrowright' onclick='openNext()' value='&#10093;'/>")

    print("</div>")
    print("</div>")

    print(f'''<script>
        const paintings = {json_data};
        let currentIndex = 0;

        function updateTable() {{
          const painting = paintings[currentIndex];
          document.getElementById("paintingYear").innerText = painting.year;
          document.getElementById("paintingName").innerText = painting.name;
          document.getElementById("paintingLink").href = painting.link;
          document.getElementById("img_full").src = painting.image;
          document.getElementById("img_full").alt = painting.alt;
          //document.getElementById("paintingID").innerText = painting.qid;
          document.getElementById("paintingSize").innerText = painting.width + ' cm x ' + painting.height + ' cm';



        }}

        function openNext() {{
          currentIndex = (currentIndex + 1) % paintings.length;
          updateTable();
        }}

        function openPrev() {{
          currentIndex = (currentIndex - 1 + paintings.length) % paintings.length;
          updateTable();
        }}

        document.addEventListener("DOMContentLoaded", updateTable);
    </script>''')




    print('<div id="infogrid"><div id="paintingName"></div>')
    print('<div id="paintingYear"></div>')
    #print('<span id="paintingSize"></span>')
    print('<span><a id="paintingLink"> </a></span></div>')









In [63]:
get_img("Q217")


<link rel='stylesheet' href='./section-fullimg.css'>
<div class='allcontent'>
<div class='navig'>

::: {.content-visible when-format='pdf'}
        

\begin{figure}[H]
\centering
\includegraphics[height=10cm]{./images/fmd10005862a.jpg}
\caption{Knight's Hall + Room 72 - to the west}
\label{fig:{https://previous.bildindex.de/bilder/fmd10005862a.jpg}}
\end{figure}

```{=latex}
\clearpage
```
        

:::
        

::: {.content-visible when-format='docx'}
        

![Knight's Hall & Room 72 - to the west](./images/fmd10005862a.jpg)
        

:::
        

::: {.content-visible when-format='pdf'}
        

\begin{figure}[H]
\centering
\includegraphics[height=10cm]{./images/fmd10005864a.jpg}
\caption{Lion couple – general view}
\label{fig:{https://previous.bildindex.de/bilder/fmd10005864a.jpg}}
\end{figure}

```{=latex}
\clearpage
```
        

:::
        

::: {.content-visible when-format='docx'}
        

![Lion couple – general view](./images/fmd10005864a.jpg)
        

:::
        

In [64]:
#get_img("")

<hr>

In [65]:
#get_graph()